## 원본

In [1]:
import requests
from bs4 import BeautifulSoup

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')
        code = stock.select_one('.tltle').attrs['href'][-6:]

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            codes.append(code)
        else:
            pass
    except:
        continue
        
    if len(codes) == 5:
        break
        
print(codes)

['102280', '001440', '034020', '014990', '009810']


## 작업 중.. 
### (codes에 담긴 종목 중 변동성 돌파 기반으로 매수해야 할 종목이 있는지 추가적으로 데이터 크롤링 후 시각화 예정)

In [3]:
import requests
from bs4 import BeautifulSoup

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

In [4]:
stocks[0].select('.tah.p11')

[<span class="tah p11 red02">
 				30
 				</span>,
 <span class="tah p11 red01">
 				+0.95%
 				</span>]

In [5]:
stocks[0].select('.tah.p11')[0].text

'\n\t\t\t\t30\n\t\t\t\t'

In [6]:
stocks[0].select('.tah.p11')[1].text

'\n\t\t\t\t+0.95%\n\t\t\t\t'

In [7]:
import re

In [8]:
a = stocks[0].select('.tah.p11')[0].text
b = stocks[0].select('.tah.p11')[1].text
re_a = re.sub('[a-zA-Z]', '', a)
re_b = re.sub('[a-zA-Z]', '', b)
print(re_a, re_b)


				30
				 
				+0.95%
				


In [9]:
Notag_a = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', a)
Notag_b = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', b)
print(Notag_a, Notag_b)

30 +0.95%


In [36]:
ccc = stocks[1].select('td.number')[3:-2]
print(ccc, stocks[1].select('td.number')[3].text, stocks[1].select('td.number')[-3].text, sep = '\n')
ccc[0].text

[<td class="number">63,913,877</td>, <td class="number">33,485</td>, <td class="number">505</td>, <td class="number">506</td>, <td class="number">1,326</td>]
63,913,877
1,326


'63,913,877'

In [35]:
pp = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*|, ', '', str(ccc[0]))
pp

'63,913,877'

In [14]:
print(type(ccc[0]))

<class 'bs4.element.Tag'>


In [15]:
result = []

for cc in ccc:
    c = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*|, ', '', str(cc))
    print(cc)
    result.append(c)
result # 거래량, 거래대금, 매수호가, 매도호가, 시가총액

<td class="number">63,913,877</td>
<td class="number">33,485</td>
<td class="number">505</td>
<td class="number">506</td>
<td class="number">1,326</td>


['63,913,877', '33,485', '505', '506', '1,326']

In [3]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []
keys = ['종목명', '현재가', '전일비', '등락률', '거래량', '시가총액']
result = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            
            code = stock.select_one('.tltle').attrs['href'][-6:]
            codes.append(code)
            
            updown = stock.select('.tah.p11')[0].text
            percentage = stock.select('.tah.p11')[1].text
            
            re_updown = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', updown)
            re_percentage = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', percentage)
            
            market = stock.select('td.number')[3].text
            volumn = stock.select('td.number')[-3].text
            
            values = [stock_n, price, re_updown, re_percentage, market, volumn]
            
            info_dict = dict(zip(keys, values))
            result.append(info_dict)
        
        else:
            pass
    except:
        continue
        
    if len(codes) == 5:
        break
        
print(codes)

for r in result:
    print()
    print(r)

['102280', '018470', '010960', '001440', '014990']

{'종목명': '쌍방울', '현재가': '497', '전일비': '8', '등락률': '-1.58%', '거래량': '17,648,147', '시가총액': '1,305'}

{'종목명': '조일알미늄', '현재가': '2360', '전일비': '155', '등락률': '+7.03%', '거래량': '17,040,239', '시가총액': '2,970'}

{'종목명': '삼호개발', '현재가': '4865', '전일비': '670', '등락률': '+15.97%', '거래량': '13,529,550', '시가총액': '1,216'}

{'종목명': '대한전선', '현재가': '1680', '전일비': '40', '등락률': '-2.33%', '거래량': '13,200,677', '시가총액': '20,907'}

{'종목명': '인디에프', '현재가': '881', '전일비': '96', '등락률': '+12.23%', '거래량': '9,651,343', '시가총액': '623'}


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://finance.naver.com/sise/sise_quant.nhn'
res = requests.get(URL)
# print(res.status_code)

soup = BeautifulSoup(res.text, 'html.parser')
stocks = soup.select('.type_2 tr')[2:]

codes = []
keys = ['종목명', '현재가', '전일비', '등락률', '거래량', '시가총액']
result = []

for stock in stocks:
    try:
        stock_n = stock.select_one('.tltle').text
        price = stock.select_one('.number').text.replace(',', '')

        if (('인버스' not in stock_n) and ('레버리지' not in stock_n)) and (int(price) <= 26000):
            
            code = stock.select_one('.tltle').attrs['href'][-6:]
            codes.append(code)
            
            updown = stock.select('.tah.p11')[0].text
            percentage = stock.select('.tah.p11')[1].text
            
            re_updown = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', updown)
            re_percentage = re.sub('\t*|\n*|(\<[^\>]*\>)|\n*\t*', '', percentage)
            
            market = stock.select('td.number')[3].text
            volumn = stock.select('td.number')[-3].text
            
            values = [stock_n, price, re_updown, re_percentage, market, volumn]
            
            info_dict = dict(zip(keys, values))
            result.append(info_dict)
            
            
        
        else:
            pass
    except:
        continue
        
    if len(codes) == 5:
        break
        
print(codes)

for r in result:
    print()
    print(r)